In [6]:
!pip install fuzzywuzzy



In [3]:
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process
import pandas as pd

In [4]:
winners_data=pd.read_csv('EC-Result.csv')
candidates_data=pd.read_excel('ADR data.xlsx')

In [5]:
winners_data['Concatenated'] = winners_data['State'] + winners_data['PC Code'].astype(str) + winners_data['Party Name']
winners_data['Concatenated'] = winners_data['Concatenated'].apply(lambda x: str(x).lower().replace(' ', '') if isinstance(x, str) else '')

candidates_data['Concatenated'] = candidates_data['State'] + candidates_data['PC No'].astype(str) + candidates_data['Party Code']
candidates_data['Concatenated'] = candidates_data['Concatenated'].apply(lambda x: str(x).lower().replace(' ', '') if isinstance(x, str) else '')



In [6]:
# Function to match and combine data using fuzzy matching
def fuzzy_match_and_combine(df1, df2, key1, key2, threshold=90):
    matched_indices = []
    # Convert the choices to a list of strings
    choices = [str(x) for x in df2[key2].tolist()]
    for idx, value in df1[key1].items():
        match = process.extractOne(str(value), choices, scorer=fuzz.ratio)
        if match and match[1] >= threshold:
            matched_idx = df2[df2[key2] == match[0]].index[0]
            matched_indices.append((idx, matched_idx))

    combined_rows = []
    for idx1, idx2 in matched_indices:
        combined_row = {**df1.loc[idx1].to_dict(), **df2.loc[idx2].to_dict()}
        combined_rows.append(combined_row)

    return pd.DataFrame(combined_rows)



In [7]:

combined_df = fuzzy_match_and_combine(winners_data, candidates_data, 'Concatenated', 'Concatenated', threshold=85)


In [8]:
len(combined_df['Constituency Name'].unique())

540

In [9]:
combined_df.head(5)

,Unnamed: 0,Year,Candidate Name,State,Constituency Name,PC Code,Party Name,EVM Votes,Postal Votes,Total Votes,...,PC No,Education,Name,Age,Gender,Party Code,Criminal Cases,No. of Cases,Total Assets,Phase
0,0,2024,BISHNU PADA RAY,ANDAMAN & NICOBAR ISLANDS,Andaman & Nicobar Islands,1,Bharatiya Janata Party,102182,254,102436,...,1,Graduate,Bishnu Pada Ray,73,M,BHARATIYA JANATA PARTY,No,0.0,27439170,1.0
1,13,2024,GUMMA THANUJA RANI,ANDHRA PRADESH,Araku,1,Yuvajana Sramika Rythu Congress Party,471470,5535,477005,...,1,Graduate Professional,Gumma Thanuja Rani,31,F,YUVAJANA SRAMIKA RYTHU CONGRESS PARTY,NaN,NaN,0.235468,4.0
2,27,2024,KINJARAPU RAMMOHAN NAIDU,ANDHRA PRADESH,Srikakulam,2,Telugu Desam,734501,19827,754328,...,2,Post Graduate,Kinjarapu Rammohan Naidu,36,M,TELUGU DESAM,NaN,NaN,23.307356,4.0
3,41,2024,APPALANAIDU KALISETTI,ANDHRA PRADESH,Vizianagaram,3,Telugu Desam,729284,13829,743113,...,3,Graduate Professional,Appalanaidu Kalisetti,50,M,TELUGU DESAM,NaN,NaN,6.236519,4.0
4,57,2024,MANISH TEWARI,Chandigarh,Chandigarh,1,Indian National Congress,216350,307,216657,...,1,Graduate Professional,Manish Tewari,58,M,INDIAN NATIONAL CONGRESS,NaN,NaN,296842688,7.0


In [13]:
combined_df

,Unnamed: 0,Year,Candidate Name,State,Constituency Name,PC Code,Party Name,EVM Votes,Postal Votes,Total Votes,...,PC No,Education,Name,Age,Gender,Party Code,Criminal Cases,No. of Cases,Total Assets,Phase
0,0,2024,BISHNU PADA RAY,ANDAMAN & NICOBAR ISLANDS,Andaman & Nicobar Islands,1,Bharatiya Janata Party,102182,254,102436,...,1,Graduate,Bishnu Pada Ray,73,M,BHARATIYA JANATA PARTY,No,0.0,27439170,1.0
1,13,2024,GUMMA THANUJA RANI,ANDHRA PRADESH,Araku,1,Yuvajana Sramika Rythu Congress Party,471470,5535,477005,...,1,Graduate Professional,Gumma Thanuja Rani,31,F,YUVAJANA SRAMIKA RYTHU CONGRESS PARTY,NaN,NaN,0.235468,4.0
2,27,2024,KINJARAPU RAMMOHAN NAIDU,ANDHRA PRADESH,Srikakulam,2,Telugu Desam,734501,19827,754328,...,2,Post Graduate,Kinjarapu Rammohan Naidu,36,M,TELUGU DESAM,NaN,NaN,23.307356,4.0
3,41,2024,APPALANAIDU KALISETTI,ANDHRA PRADESH,Vizianagaram,3,Telugu Desam,729284,13829,743113,...,3,Graduate Professional,Appalanaidu Kalisetti,50,M,TELUGU DESAM,NaN,NaN,6.236519,4.0
4,57,2024,MANISH TEWARI,Chandigarh,Chandigarh,1,Indian National Congress,216350,307,216657,...,1,Graduate Professional,Manish Tewari,58,M,INDIAN NATIONAL CONGRESS,NaN,NaN,296842688,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,8736,2024,KUNDURU RAGHUVEER,TELANGANA,Nalgonda,13,Indian National Congress,772264,12073,784337,...,13,NaN,Kunduru Raghuveer,44,M,INDIAN NATIONAL CONGRESS,NaN,NaN,35.57315,4.0
539,8759,2024,CHAMALA KIRAN KUMAR REDDY,TELANGANA,Bhongir,14,Indian National Congress,623762,5381,629143,...,14,NaN,Chamala Kiran Kumar Reddy,47,M,INDIAN NATIONAL CONGRESS,NaN,NaN,28.721147,4.0
540,8799,2024,KADIYAM KAVYA,TELANGANA,Warangal,15,Indian National Congress,575566,5728,581294,...,15,Post Graduate,Kadiyam Kavya,41,F,INDIAN NATIONAL CONGRESS,NaN,NaN,1.550103,4.0
541,8842,2024,BALRAM NAIK PORIKA,TELANGANA,Mahabubabad,16,Indian National Congress,608505,4269,612774,...,16,NaN,Balram Naik Porika,60,M,INDIAN NATIONAL CONGRESS,NaN,NaN,2.99191,4.0
